# Solving Optimization Problem with Qiskit

## Team name : SO(3)
It means ...

### Team members
Kim BoSeong

Choi KwangJun

Kim HeeKang

Choi JaeWon

## Delivery Reward Maximization
__Delivery Reward Maximization(DRM)__ is a optimization problem. Suppose there are customers who want your goods. But you have limited time. Customers don't wait forever. Then how can you maximize your profit? __DRM__ will give you an answer.

Before telling what __DRM__ is, define the problem mathematically. Let $V=\{v_1,v_2,\cdots,v_n\}$ be the nodes, the customers, and $v_0$ be your store. Each customer has deadline $d(v_i)$ and reword $c(v_i)$. Your vehicle's speed is $s$. Path is denoted as edge. Weights of egdes $w(e_i,e_j)$ represent distance. There is also service time $t(v_i)$ that may represent the time needed to pass the goods. __DRM__ algorithm is hard. In this project, we consider a special case called __LDRM(Line Delivery Reward Maximization)__.

We modified __LDRM__ little. Assume that $v_0$ is at origin and customers were numbered sequentially, that is, nodes will like this $v_0,v_1,v_2,\cdots$. We added varibales $x_{11},x_{22},\cdots$ which represent whether the solution ends at $v_i$ or not. Also set variables $x_{ij}$, which implies that airplane stop by $j$th airport whose final destination is $i$. We added a constraint that $\sum\limits_{j<i} x_{ij}-ix_{ii}\leq 0$. Note that $x_{ii}$ is 1 if and only if the final destination is $i$. Then there are $i$ possible airport that airplane can stop by. So when $x_{ii}=0$, $x_{ij}$ is 0 for all $j$. Otherwise, airplane can choose airport before its destination.

## Into a real problem
Suppose the depot is in Incheon. You may deliver you goods by airplane. Since your goods require extremly cold environment, there is few airplane that can carry your goods. In this case, assume that only one airplane is available. Customers are in Shanghai, Hanoi and Suvarnabhmi. Roughly speaking, they are in a line fortunately. So this problem can be converted into __LDRM__.

In [2]:
#Import packages to visualization
from ipyleaflet import Map, AntPath, Marker
from itertools import combinations

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'ipyleaflet'

In [46]:
#Instanciate map
m = Map(center=(25.500, 122.037), zoom=3)

#Longitude and latitude of airports
locations = {"INCHEON" : (37.46035,126.44061),"HANOI" : (21.21791,105.78959),"SHANGHAI" : (31.19992,121.33644),"SUVARNABHMI":(13.69036,100.74884)}
for location in locations.values():
    m.add_layer(Marker(location=location, draggable=False));
    
#Add pathes
for twoPoints in combinations(locations.values(),2):
    m.add_layer(AntPath(locations=
            [(twoPoints[0][0]+(twoPoints[1][0]-twoPoints[0][0])*i/10,twoPoints[0][1]+(twoPoints[1][1]-twoPoints[0][1])*i/10 ) for i in range(10)]
            ,dash_array=[1, 10],delay=1000,color='#7590ba',pulse_color='#3f6fba'))

m

Map(center=[25.5, 122.037], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [ ]:
#Import packages about qiskit
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

In [ ]:
#Final code

## Conclusion
There are many __NP-HARD__ problems in real world. Any computer, even a supercomputer, can solve these problems. It is impossible to get exact solution. But if we don't have to get the best solution, if we can be satisfied with optimized solution, we can get a solution faster. Still, for large data, some optimization algorithm are slow. In this case quantum computing can be a solution. Using quantum speedup, one can get optimized solution in reasonable time.

## Appendix : DRM is NP-HARD
We will follow the step that the paper shows. Suppose there is no weight and deadline is constant. Then it is a special case of __DRM__. This problem is described as

Maximize $c(v_i)$

Subject to

$\sum t(v_i) \leq D$

Remember knapsack problem. The bag has limited weight and you want to maximize value.

Maximize $v(x_i)$

Subject to

$\sum w(x_i) \leq W$

As you can check two are equivalent. Since __knapsack problem__ is __NP-HARD__ problem, so is __DRM__.

In [66]:
from docplex.mp.model import Model
model = Model(name='knapsack')

#Define the number of items
numOfItem=5

#Define variable which represent whether item is in bag or not.
isUsed = [model.binary_var(name='Item #'+str(i)) for i in range(numOfItem)]

#Weight and cost for each item
weight = [40,90,120,100,200]
cost = [40,25,40,20,60]

weightLimit = 300

model.add_constraint(sum([isUsed[i]*weight[i] for i in range(numOfItem)])-weightLimit<=0,'Weight limit')
model.maximize(sum([isUsed[i]*cost[i] for i in range(numOfItem)]))
print(model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: knapsack

Maximize
 obj: 40 Item_#0 + 25 Item_#1 + 40 Item_#2 + 20 Item_#3 + 60 Item_#4
Subject To
 Weight_limit: 40 Item_#0 + 90 Item_#1 + 120 Item_#2 + 100 Item_#3 + 200 Item_#4
                <= 300

Bounds
 0 <= Item_#0 <= 1
 0 <= Item_#1 <= 1
 0 <= Item_#2 <= 1
 0 <= Item_#3 <= 1
 0 <= Item_#4 <= 1

Binaries
 Item_#0 Item_#1 Item_#2 Item_#3 Item_#4
End



In [67]:
def compareQE(model):
    aqua_globals.random_seed = 10598
    quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
    qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    qaoa_result = qaoa.solve(mod)
    print('QAOA result')
    print(qaoa_result)
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)
    exact_result = exact.solve(mod)
    print('Exact result')
    print(exact_result)

In [68]:
#Change to use qiskit
mod = QuadraticProgram()
mod.from_docplex(model)
compareQE(mod)

QAOA result
optimal function value: 105.0
optimal value: [1. 1. 1. 0. 0.]
status: SUCCESS
Exact result
optimal function value: 105.0
optimal value: [1. 1. 1. 0. 0.]
status: SUCCESS


## Reference
[1] Chuanwen Luo, Deying Li, Xingjian Ding, Weili Wu, __*Delivery Route Optimization with automated vehicle in smart urban environment*__, Theoretical Computer Science 836(2020), 42-52